In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb

# Tokenizer 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

# 질의를 임베딩으로 변환하는 함수 (mean pooling 포함)
def tokenize_and_pooling(model, tokenizer, text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    attention_mask = encoded_input['attention_mask']
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

c:\Users\MoonDongWon\OneDrive - gnu.ac.kr\바탕 화면\프로젝트\과제 도우미\report_helper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\MoonDongWon\OneDrive - gnu.ac.kr\바탕 화면\프로젝트\과제 도우미\report_helper\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
client = chromadb.PersistentClient('../../chroma')

collection_name = "paper_embeddings"
try:
    paper_embeddings = client.get_collection(name=collection_name)
except Exception as e:
    print(e)
    print('올바른 경로 설정 필요')

In [3]:
def search(query: str):
    query_text = query
    query_embedding = tokenize_and_pooling(model, tokenizer, query_text)
    results = paper_embeddings.query(
        query_embeddings=query_embedding.numpy().tolist(),
        n_results=5
    )
    for result in results['metadatas'][0]:
        print(result['title'], '\n')

In [7]:
search('SNS의 사용자 관계 구조 및 메시지를 분석한 연구가 있는 논문을 추천해줘')

SNS에서의 개선된 소셜 네트워크 분석 방법 

토픽 모델링을 이용한 트위터 이슈 트래킹 시스템 

웹 기반 소셜 네트워크에서 시맨틱 관계 추론 및 시각화 

텍스트 마이닝을 이용한 감정 유발 요인 ‘Emotion Trigger’에 관한 연구 

사회적 감성이 가상화폐 가격에 미치는 영향분석  실물화폐와의 비교를 중심으로 

